In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json
import psycopg2
from sodapy import Socrata
from transform_dag import transformations_api

In [4]:
def read_db_water():
    with open('db_config.json') as file:
        db_config = json.load(file)

    engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5433/{db_config["dbname"]}')

    water = pd.read_sql('SELECT * FROM water_table', engine)
    
    return water

In [6]:
from transform_dag import transformations_water

def transform_water():
    
    water = read_db_water()
    
    water = transformations_water(water)
    
    print(water.head())

# Llamar a la función principal para procesar los datos
transform_water()



OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [2]:
def extract_api(endpoint):
    try:
        # Inicializa el cliente de Socrata sin token de aplicación
        client = Socrata("www.datos.gov.co", None)

        # Obtiene los datos desde el endpoint especificado
        results = client.get(endpoint, limit=2000)

        # Convierte los resultados en un DataFrame
        api = pd.DataFrame.from_records(results)

        return api
    except Exception as e:
        print(f"Se produjo un error: {e}")

In [3]:
api = extract_api("tcwu-r53g")
print(api.head())

2024-05-06 13:53:34,873 - WARNING - Requests made without an app_token will be subject to strict throttling limits.


  fecha_terminacion_proyecto           fecha_de_corte  \
0    2018-12-31T00:00:00.000  2024-04-04T00:00:00.000   
1    2018-10-16T00:00:00.000  2024-04-04T00:00:00.000   
2    2018-10-08T00:00:00.000  2024-04-04T00:00:00.000   
3    2019-06-01T00:00:00.000  2024-04-04T00:00:00.000   
4    2019-02-12T00:00:00.000  2024-04-04T00:00:00.000   

  c_digo_divipola_departamento departamento c_digo_divipola_municipio  \
0                           18      CAQUETA       18029, 18205, 18610   
1                           15       BOYACA                     15693   
2                           17       CALDAS                     17050   
3                           17       CALDAS                     17446   
4                           05    ANTIOQUIA                      5250   

                                           municipio  \
0  Albania(CAQ), Curillo(CAQ), San Jose De Fragua...   
1                         Santa Rosa De Viterbo(BOY)   
2                                      Aranzazu(CA

In [3]:

def transform_api():
    
    pd.set_option('display.max_columns', None)  # Ninguna limitación en el número de columnas a mostrar
    pd.set_option('display.max_rows', None)  # Ninguna limitación en el número de filas a mostrar
    
    api = extract_api("tcwu-r53g")
    
    api = transformations_api(api)
    
    print(api.head())

transform_api()

2024-05-11 10:52:47,124 - WARNING - Requests made without an app_token will be subject to strict throttling limits.
2024-05-11 10:52:59,261 - INFO - Starting transformations on API data.
2024-05-11 10:52:59,275 - INFO - Elimination of parentheses within municipalities successfully.
2024-05-11 10:52:59,293 - INFO - Separates records with multiple municipalities into individual rows successfully.
2024-05-11 10:52:59,296 - INFO - Elimination of extra spaces and capitalization of each municipality name successful.
2024-05-11 10:52:59,313 - INFO - Dates converted successfully.
2024-05-11 10:52:59,327 - INFO - Text columns normalized successfully.
2024-05-11 10:52:59,330 - INFO - Number of municipalities computed successfully.
2024-05-11 10:52:59,337 - INFO - Regions mapped successfully.
2024-05-11 10:52:59,339 - INFO - Project financing calculated successfully.
2024-05-11 10:52:59,343 - INFO - Project duration calculated successfully.
2024-05-11 10:52:59,347 - INFO - Unnecessary columns dro

  fecha_terminacion_proyecto c_digo_divipola_departamento departamento  \
0                 2018-12-31                           18      CAQUETA   
0                 2018-12-31                           18      CAQUETA   
0                 2018-12-31                           18      CAQUETA   
1                 2018-10-16                           15       BOYACA   
2                 2018-10-08                           17       CALDAS   

  c_digo_divipola_municipio              municipio  \
0       18029, 18205, 18610                albania   
0       18029, 18205, 18610                curillo   
0       18029, 18205, 18610     san jose de fragua   
1                     15693  santa rosa de viterbo   
2                     17050               aranzazu   

                                           indicador  \
0       nueva población beneficiada acueducto urbano   
0       nueva población beneficiada acueducto urbano   
0       nueva población beneficiada acueducto urbano   
1     

In [8]:
# Cargar los datasets
api_done_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/ETL-Proyect/Main_Aqua_Quality/API/DATA/api_done.csv')

water_cleaned_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/water_cleaned.csv')


# Convertir las columnas de fecha a formato datetime
api_done_df['fecha_terminacion_proyecto'] = pd.to_datetime(api_done_df['fecha_terminacion_proyecto'])
water_cleaned_df['año'] = pd.to_datetime(water_cleaned_df['año'])

# Filtrar el dataset de calidad de agua para incluir solo los años 2018 y 2019
water_filtered_df = water_cleaned_df[(water_cleaned_df['año'].dt.year == 2018) | (water_cleaned_df['año'].dt.year == 2019)]

# Realizar un merge asof
merged_df = pd.merge_asof(
    water_filtered_df.sort_values('año'),
    api_done_df.sort_values('fecha_terminacion_proyecto'),
    left_on='año',
    right_on='fecha_terminacion_proyecto',
    left_by='nombre_municipio',
    right_by='municipio',
    direction='backward'
)

# Reemplazar valores NaN en las columnas relacionadas con proyectos con "Ausencia de proyecto"
columns_project_related = ['indicador', 'nombre_proyecto', 'origen', 'aporte_nacion', 'contrapartida',
                           'estado_seguimiento', 'num_municipios', 'región', 'total_financiamiento', 
                           'duracion_proyecto_dias']
for column in columns_project_related:
    merged_df[column].fillna("Ausencia de proyecto", inplace=True)

# Mostrar los primeros registros del DataFrame resultante
print(merged_df.head())
# Guardar el DataFrame resultante a un archivo CSV
merged_df.to_csv('nuevo_merge', index=False)



C:\Users\JSLV3\AppData\Local\Temp\ipykernel_15184\718456096.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[column].fillna("Ausencia de proyecto", inplace=True)
C:\Users\JSLV3\AppData\Local\Temp\ipykernel_15184\718456096.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Ausencia de proyecto' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_df[column].fillna("Ausencia de proyecto"

         año nombre_departamento  div_dpto nombre_municipio  divi_muni  \
0 2018-01-01           Antioquia         5          Guatape       5321   
1 2018-01-01        Cundinamarca        25          Fomeque      25279   
2 2018-01-01        Cundinamarca        25          Fomeque      25279   
3 2018-01-01        Cundinamarca        25          Fomeque      25279   
4 2018-01-01        Cundinamarca        25          Fomeque      25279   

   irca_minimo  irca_maximo  irca_promedio nombre_parametro_analisis  \
0          0.0        26.54           0.74         Alcanilidad Total   
1          0.0        54.54          11.41                 Mesofilos   
2          0.0        54.54          11.41                  Mercurio   
3          0.0        54.54          11.41                 Manganeso   
4          0.0        54.54          11.41                  Magnesio   

   numero_parametros_promedio   rango_irca tratamiento_categoría  \
0                           4   Sin riesgo  Tratamient

In [13]:


# Cargar los datasets


api_done_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/ETL-Proyect/Main_Aqua_Quality/API/DATA/api_done.csv')

water_cleaned_df = pd.read_csv('C:/Users/JSLV3/Documents/5to Semestre/ETL/Proyecto Water Quality/water_cleaned.csv')


# Convertir las columnas de fecha a formato datetime
api_done_df['fecha_terminacion_proyecto'] = pd.to_datetime(api_done_df['fecha_terminacion_proyecto'])
water_cleaned_df['año'] = pd.to_datetime(water_cleaned_df['año'])

# Filtrar el dataset de calidad de agua para incluir solo los años 2018 y 2019
water_filtered_df = water_cleaned_df[(water_cleaned_df['año'].dt.year == 2018) | (water_cleaned_df['año'].dt.year == 2019)]

# Realizar un merge asof, asegurando que los nombres de los municipios coincidan
merged_df = pd.merge_asof(
    water_filtered_df.sort_values('año'),
    api_done_df.sort_values('fecha_terminacion_proyecto'),
    left_on='año',
    right_on='fecha_terminacion_proyecto',
    left_by='nombre_municipio',
    right_by='municipio',
    direction='backward'
)

# Reemplazar valores NaN en las columnas relacionadas con proyectos
columns_project_numeric = ['aporte_nacion', 'contrapartida', 'num_municipios', 'total_financiamiento', 'duracion_proyecto_dias']
columns_project_text = ['indicador', 'nombre_proyecto', 'origen', 'estado_seguimiento', 'región']

# Asignar -1 para columnas numéricas y 'Ausencia de proyecto' para columnas de texto
for column in columns_project_numeric:
    merged_df[column] = merged_df[column].fillna(-1)

for column in columns_project_text:
    merged_df[column] = merged_df[column].fillna("Ausencia de proyecto")

# Guardar el DataFrame resultante a un archivo CSV
merged_df.to_csv('merged_dataset.csv', index=False)
